# Restaurant Demo: Creating Differentially Private Business Stats

PyDP library version for Google's Java Differential Privacy Library example - https://github.com/google/differential-privacy/tree/master/examples/java.

***Note: This demo demonstrates calculation of both the non-private and private results i.e., without using PyDP library and when using PyDP library, for better comparison and understanding of results.***

Imagine a fictional restaurant owner named Alice who would like to share business statistics with her visitors or potential clients. Alice knows when visitors enter the restaurant and how much time and money they spend there. To ensure that visitor's *privacy is preserved*, Alice decides to use the **Differential Privacy library** in this case PyDP library. 

Alice wants to share the information with potential clients which includes 3 cases in total.

1. **Count visits by hour of day**: Count how many visitors enter the restaurant at every hour of a particular day.
2. **Count visits by day of week**: Count how many visitors enter the restaurant each day in a week.This in itself includes Two cases- 
        (i) Case 1 - Visitors can enter the restaurant once a day and multiple times in a week.
        (ii) Case 2 - Visitors can enter the restaurant once a day and max three times in a week.       
3. **Sum-up revenue per day of the week**: Calculate the sum of the restaurant revenue per week day.     
    
Both the Non-private and the Private values for the above cases are calculated down below along with the graphical representation of the results and their comparisons.  

Let's begin!!



## Initial Setup

In [ ]:
# Install the PyDP package
! pip install python-dp

In [ ]:
import pydp as dp  # by convention our package is to be imported as dp (for Differential Privacy!)
from pydp.algorithms.laplacian import BoundedSum, Count
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

## 1. Count visits by hour of day

In this example Alice wants to share this information with potential clients in order **to let them know the busiest times in the restaurant**. For this, you will ***count how many visitors enter the restaurant at every hour of a particular day***. For simplicity, assume that a visitor comes to the restaurant at most once a day. In other words, a visitor is present at most once in the whole dataset.

Visit data for a single day is stored in the **day_data.csv** file. It includes the visitor’s ID, the visit duration (in minutes), and the money spent at the restaurant.

## Restaurant Schedule

In [ ]:
# An hour when visitors start entering the restaurant.
OPENING_HOUR = 9

# An hour when visitors stop entering the restaurant.
CLOSING_HOUR = 20

# For how many hours visitors can enter the restaurant.
NUM_OF_WORK_HOURS = CLOSING_HOUR - OPENING_HOUR + 1  # 12

# Range of valid work hours when a visitor can enter the restaurant.
VALID_HOURS = list(
    range(OPENING_HOUR, CLOSING_HOUR + 1)
)  # [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

Get and view the Visit data for a single day 

In [ ]:
url = "https://raw.githubusercontent.com/OpenMined/PyDP/dev/examples/Tutorial_2-restaurant_demo/day_data.csv"
df = pd.read_csv(url, sep=",")
df.head()

You want to read the daily statistics and calculate the number of *visitors* that entered the restaurant *every hour* of the day. 

In [ ]:
# Get the VisitorId and time entered data into new DataFrame
cols = ["VisitorId", "Time entered"]
new_df = df.loc[:, cols]
new_df.head()

If you have noticed the "Time entered" is in 12 hours format. The restaurant is opened at 9 am and closed at 8 pm, i.e., visitors can enter the restaurant during these 11 working hours anytime between 9 am and 8 pm . The resulting aggregated data is split into partitions, 12 partitions, one for each entry hour. For simplicity, convert_to_24hours() is used to convert the time from 12 hours format to 24 hours format as shown below. 

In [ ]:
# to check sample data for time
time = df.iloc[421, 1]
time

In [ ]:
# function to convert time to 24 hrs format


def convert_to_24hours(date):
    dateInfo = date.split(" ")  #  ['8:09:00', 'PM']
    dateArr = dateInfo[0].split(":")  # ['8', '09', '00']
    hour = 0
    # for midnight
    if (dateInfo[1] == "AM") and (dateArr[0] == "12"):
        hour = 0
    # for 12 noon
    elif (dateInfo[1] == "PM") and (dateArr[0] == "12"):
        hour = 12
    # for AM and PM
    elif dateInfo[1] == "AM":
        hour = int(dateArr[0])
    else:
        hour = int(dateArr[0]) + 12
    return hour


# check
hour24 = convert_to_24hours("1:23:34 PM")
print(hour24)

## Explanation for "Count visits by hour of day"
### Partitions and contributions
The resulting aggregated data is split into partitions. The bar chart for the private and non-private counts each have 12 partitions, one for each entry hour.

More generally, a single partition represents a subset of aggregated data corresponding to a given value of the aggregation criterion. Graphically, a single partition is represented as a bar on the aggregated bar chart.

A visitor contributes to a given partition if their data matches the partition criterion. For example, if a visitor enters between 8 AM and 9 AM, they contribute to the 8 AM partition.

Recall that in the the example above, a visitor can enter the restaurant **only once per day**. This implies three contribution bounds:

- **Maximum partitions contributed**: To how many partitions can a visitor contribute? In this example,a visitor can contribute up to one partition. In other words, there is at most one time-slot when a visitor with a given id can enter the restaurant.
- **Maximum contributed value**: What is the maximum value that can be contributed by a visitor to a partition? In this example, the number of visits are counted, so the maximum contributed value is simply 1.
- **Maximum contributions per partition**: How many times can a visitor contribute to a partition? In this example, a visitor can contribute to a partition at most once. In other words, a visitor can enter the restaurant only once at a given hour.

Why is this important? Differential Privacy adjusts the amount of noise to mask contributions of each visitor. More contributions require more noise.

In [ ]:
# Create a new column called partition with hour data in the new_df
# if a visitor enters between 8 AM and 9 AM, they contribute to the 8 AM partition.
# Note: for simplicity the time is converted to 24 hours format.

partition = []

for time in df["Time entered"]:  # convert timings into 24hrs format
    new = convert_to_24hours(time)
    partition.append(new)


new_df["Partition"] = partition  # add new column
new_df.tail()

### Defining Bar Plots for Original Data, Differentially Private data and their comparison

In [ ]:
# bar plot - original data


def bar_plot(x, y, title, x_label, y_label):
    plt.subplots(figsize=(8, 4))
    plt.bar(x, y, color="cornflowerblue")
    axes = plt.gca()
    axes.yaxis.grid()
    axes.set_axisbelow(True)

    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.xticks(x)
    plt.show

In [ ]:
# Bar plot - differentially private data


def private_bar_plot(x, y, title, x_label, y_label):
    plt.subplots(figsize=(8, 4))
    plt.bar(x, y, color="orangered")
    axes = plt.gca()
    axes.yaxis.grid()
    axes.set_axisbelow(True)

    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.xticks(x)
    plt.show

In [ ]:
# Comparison Bar plot


def comparison_graph(labels, x1, x2, rect1_label, rect2_label, x_label, title):
    x = np.arange(len(labels))  # the label locations
    width = 0.34  # the width of the bars

    fig, ax = plt.subplots(figsize=(14, 6))
    rects1 = ax.bar(
        x - width / 2,
        x1,
        width,
        label=rect1_label,
        color="cornflowerblue",
        edgecolor="white",
    )
    rects2 = ax.bar(
        x + width / 2,
        x2,
        width,
        label=rect2_label,
        color="orangered",
        edgecolor="white",
    )
    axes = plt.gca()
    axes.yaxis.grid()
    axes.set_axisbelow(True)

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(x_label)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = rect.get_height()
            ax.annotate(
                "{}".format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 5),  # 3 points vertical offset
                textcoords="offset points",
                ha="center",
                va="bottom",
            )

    autolabel(rects1)
    autolabel(rects2)

    fig.tight_layout()

    plt.show()

## Count Per Hour  (without using PyDP library)

In [ ]:
# visitor contributes to a given partition's VALID_HOURS
# In the given data 1 visitor contributes to only one partition as suggested by 'Maximum contributions per partition'

# Count the no of visitors per hour
count = new_df.groupby("Partition").VisitorId.count()  # returns a series
# count

# convert series to list
count_per_hour = list(count)
print(count_per_hour)

In [ ]:
# bar_plot(x,y,title,x_label,y_label)

bar_plot(
    VALID_HOURS,
    count_per_hour,
    "Visitors Count per hour",
    "Partitions - Open hours",
    "Visitors",
)

### Note: The data preprocessing is done by the user, the library PyDP only helps with introducing noise to the data and making it differentially private

### Private Count Per Hour (Using PyDP library)

Introducing noise to the data, making it differentially private.

In [ ]:
data = new_df.groupby("Partition").VisitorId.apply(list)
data_list = list(data)  # list

In [ ]:
# Private Count visits by hour of day

private_count_per_hour = []
epsilon = math.log(3)


def cal_private_count(privacy_budget: float):
    for data in data_list:
        x = Count(epsilon=privacy_budget)
        count = x.quick_result(data)  # accepts only list as input
        private_count_per_hour.append(count)
    return private_count_per_hour


cal_private_count(epsilon)
print(private_count_per_hour)

In [ ]:
# private_bar_plot(x,y,title,x_label,y_label)

private_bar_plot(
    VALID_HOURS,
    private_count_per_hour,
    "Differentially private - Visitors Count per hour",
    "Partitions - Open hours",
    "Visitors",
)

The image below illustrates the results. The blue (left) bars represent the counts without anonymization while orange(right) bars correspond to the private (or anonymized) counts. You can see that the private values slightly differ from the actual ones but the overall trend is preserved. For example, you can clearly see that the restaurant is more busy during lunch and dinner time.

In [ ]:
# compare the results
labels = VALID_HOURS
x1 = count_per_hour
x2 = private_count_per_hour

# comparison_graph(labels, x1, x2, rect1_label, rect2_label, x_label, title)
comparison_graph(
    labels,
    x1,
    x2,
    "CountPerHour",
    "PrivateCountPerHour",
    "Visitors",
    "Visitors Count per hour VS Differentially private - Visitors Count per hour",
)

Note that Differential Privacy involves adding random noise to the actual data, so your results will most likely be slightly different.

 ### Next, demonstrating how to use the PyDP library in scenarios where: 

- ***Visitors can contribute to multiple partitions.***
- ***Contributed values can be greater than 1.***
- ***Visitors can contribute to a partition multiple times.***

## 2. Count visits by day of week 

The previous example made some over-simplifying assumptions. Now, let’s have a look at the use-case where visitors can contribute to multiple partitions.

Imagine Alice decides to let visitors know which days are the busiest at her restaurant. For this, she calculates how many people visit the restaurant every day of the week. For simplicity, let’s assume a visitor enters the restaurant at most once a day but multiple times a week.

Visit data for a week is stored in the **week_data.csv** file.

- **Case i** ***-  once a day - multiple times a week ***
- **Case ii** ***- once a day - max three times a week***

Speaking in terms of partitions and contributions, the resulting bar chart has **7 partitions:** one for each day of the week. A visitor may enter the restaurant **once a day** and hence contribute to a partition at most once. A visitor may enter the restaurant **several times a week** and hence contribute to up to 7 partitions.

## Case i : Max contribution per week set to 7 times a week

Get and View Visit data for a week on per day basis.

In [ ]:
url = "https://raw.githubusercontent.com/OpenMined/PyDP/dev/examples/Tutorial_2-restaurant_demo/week_data.csv"
visitor_df = pd.read_csv(url, sep=",")
visitor_df.head()

In [ ]:
# check the data
visitor_df[["VisitorId", "Day"]].head()

In [ ]:
# Day for index - check
print(visitor_df.loc[2849, "Day"])

In [ ]:
# view and check
# Visitors in Day -1
day1 = visitor_df.loc[visitor_df["Day"] == 1]
print(day1)

# Search Visitor ID details
visitor = visitor_df.loc[visitor_df["VisitorId"] == 1215]
print(visitor)

# Count number of days visit per week > 3
print(visitor["Day"].count() > 3)

# combining the two - visit count > 3per week
print(visitor_df.loc[visitor_df["VisitorId"] == 1215]["Day"].count() > 3)

# days of visit
visitor_day = visitor_df.loc[visitor_df["VisitorId"] == 1215]["Day"]
print(visitor_day)

Defining variables for Day count, day label and day number for various purposes.

In [ ]:
day = [{"Day_" + str(i): 0} for i in visitor_df["Day"].unique()]
print(day)

days = ["Day_" + str(i) for i in visitor_df["Day"].unique()]
print(days)

days_int = [i for i in visitor_df["Day"].unique()]
print(days_int)

#### Steps to Calculate Non-private visit count per day in a week
1. Create **new dataframe** to store the per day visit values.
2. Set the initial count for each day visit to "0" and valid_contribution to "False" using **set_visitor_count0()**
3. Add per day visits using **per_day_visits()** where '1' and '0' represent present and absent respectively.
4. For case 2 - Using **max_partitions_contributed()** change valid_contribution to "True" is maxPartitionsContributed is equal to 3.
5. Remove extra contributions using **remove_extra_contributions()**
6. Get the day wise count for both the cases and plot it on bar graph.


In [ ]:
# Another DataFrame for maxPartitionsContributed per day
visitor_count_df = pd.DataFrame({"VisitorId": visitor_df["VisitorId"].unique()})
# print(visitor_count)

# Set all Days visit to 0 and Count_perWeek (<= 3) to False
def set_visitor_count0(df):
    for i in days:
        df[i] = 0

    df["valid_contribution"] = False
    return df

In [ ]:
# Reset the count of day visit to 0

set_visitor_count0(visitor_count_df).head()

In [ ]:
# Add Per Day visit to visitor_count_df
# 1 - present , 0 - absent


def per_day_visits(df):
    for index_no in visitor_df.index:
        day_no = visitor_df.loc[index_no, "Day"]
        id_no = visitor_df.loc[index_no, "VisitorId"]
        # index_val = visitor_df.loc[visitor_df["VisitorId"] == id_no].index[0]
        # print("index val : ",index_val, "day_no :", day_no)

        # in visitor_count_df dataframe
        visitor_index_val = df.loc[df["VisitorId"] == id_no].index[0]
        # print("New index val : ",visitor_index_val)
        visitor_day_val = df.loc[visitor_index_val, "Day_" + str(day_no)]
        # print("Day val : ",visitor_day_val)
        df.at[visitor_index_val, "Day_" + str(day_no)] = visitor_day_val + 1

    return df

In [ ]:
per_day_visits(visitor_count_df).head()

Verify the results

In [ ]:
# Day visits for VisitorId
visitor_df.loc[visitor_df["VisitorId"] == 1251][
    "Day"
]  # for this example -> Day_3 and Day_6

In [ ]:
# verifying the results for per_day_visits()
visitor_count_df.loc[visitor_count_df["VisitorId"] == 1251]

In [ ]:
# valid_contribution is all set to False
visitor_count_df.loc[visitor_count_df["valid_contribution"] == True]["Day_6"]

## Bounding the number of contributed partitions

The parameter maxPartitionsContributed in Google's Java example, defines the maximum number of partitions a visitor may contribute to. You might notice that the value of maxPartitionsContributed in the example is 3 instead of 7. Why is that? Differential Privacy adds some amount of random noise to hide contributions of an individual. The more contributions an individual has, the larger the noise is. This affects the utility of the data. In order to preserve the data utility, an approximate estimate of how many times a week a person may visit a restaurant on average was made, and assumed that the value is around 3 instead of scaling the noise by the factor of 7.

**max_partitions_contributed()** limits the max contribution to the entered number, in this case 3, i.e., a visitor may contribute or visit the restaurant max 3 times in a week. The rest of the exceeding data is discarded. Everyone with contributions greater than 3 will have **"valid_contribution"** as False.


## Case ii : Max contribution per week set to 3 times a week

In [ ]:
# Check the visitors with greater than Max Contribution are marked as False


def max_partitions_contributed(max_days: int, df):
    for index_no in df.index:
        day_count = 0
        for day_no in days_int:
            if df.loc[index_no, "Day_" + str(day_no)] > 0:
                day_count = day_count + 1

        # check whether day count is > max_days or not
        if day_count <= max_days:
            df.at[index_no, "valid_contribution"] = True
    return df

In [ ]:
# max_partitions_contributed(max_days: int, df)
max_partitions_contributed(3, visitor_count_df).head()

In [ ]:
# create a copy visitors_maxweek DataFrame for max 3 days count
visitors_maxweek = visitor_count_df.copy()
visitors_maxweek.head()

In [ ]:
# check index
visitors_maxweek.loc[visitor_count_df["valid_contribution"] == False].index

In [ ]:
# If Max contribution is greater than the desired value here 3, then,
# remove the extra contributions from that visitors per day of week
# set the "valid_contribution" to True after the extra value has been removed.
# This might introduce some bias as when max_contribution = 3, the first 3 days will remain mostly unchanged
# Day_4 to Day_7 count values will be effected the most.
# But, for simplicity this approach will be used.


def remove_extra_contributions(max_contribution: int, df):

    for index_no in df.loc[df["valid_contribution"] == False].index:
        day_count = 0
        for day_no in days_int:
            if df.loc[index_no, "Day_" + str(day_no)] > 0:
                day_count = day_count + 1

            if day_count > max_contribution:
                df.at[index_no, "Day_" + str(day_no)] = 0

        df.at[index_no, "valid_contribution"] = True

    return df

In [ ]:
remove_extra_contributions(3, visitors_maxweek).head()

In [ ]:
# check index
visitors_maxweek.loc[visitors_maxweek["valid_contribution"] == False].index

## Day wise count for both the cases

In [ ]:
# Calculate the day wise count for both the cases
print(days)
total_visitors_max3days = []  # case 2 : for max 3 days
total_visitors_day = []  # case 1: for all 7 days

for i in days:
    total_visitors_max3days.append(
        visitors_maxweek.loc[visitors_maxweek[i] > 0][i].count()
    )  # case 2
    total_visitors_day.append(
        visitor_count_df.loc[visitor_count_df[i] > 0][i].count()
    )  # case 1


print("Case 1: ", total_visitors_day)  # for all 7 days
print("Case 2: ", total_visitors_max3days)  # for max 3 days

### Bar Plots for Case i 
Visitors can visit one time a day and any day in a week, i.e., max 7 days in a week.

In [ ]:
# bar_plot(x,y,title,x_label,y_label)
x = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

bar_plot(x, total_visitors_day, "Visitor count per Day", "Week Days", "Visitors")

### Bar Plots for Case ii 
Visitors can visit one time a day and max 3 days in a week.

In [ ]:
# bar_plot(x,y,title,x_label,y_label)
x = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

bar_plot(
    x,
    total_visitors_max3days,
    "Visitor count per Day : Max 3 times a week",
    "Week Days",
    "Visitors",
)

# Private Count for the above Cases using PyDP library

In [ ]:
def cal_private_count_per_day(
    privacy_budget: float, data_list: list
):  # using PyDP library
    private_count = []
    for data in data_list:
        x = Count(epsilon=privacy_budget)
        count = x.quick_result(data)
        private_count.append(count)
    return private_count

In [ ]:
visits_case1 = []  # list for case 1
visits_case2 = []  # list for case 2


for i in days:
    # Case 1: List for max contribution 7 days a week
    visits_case1.append(visitor_count_df.loc[visitor_count_df[i] > 0][i].tolist())

    # Case 2: List for Max contribution 3 days a week
    visits_case2.append(visitors_maxweek.loc[visitors_maxweek[i] > 0][i].tolist())

In [ ]:
# caluculate the private count
epsilon = math.log(3)
private_count_perDay_week = cal_private_count_per_day(epsilon, visits_case1)
private_count_max3Day_week = cal_private_count_per_day(epsilon, visits_case2)

print(days)
print("Case 1: Private Count per day in a week")
print(private_count_perDay_week)
print("Case 2: Private Count Max 3 days in a week")
print(private_count_max3Day_week)

In [ ]:
# private_bar_plot(x,y,title,x_label,y_label)
x = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

private_bar_plot(
    x,
    private_count_perDay_week,
    "CASE 1: Differentially private - Visitors Count per Day",
    "Week days",
    "Visitors",
)

In [ ]:
# private_bar_plot(x,y,title,x_label,y_label)
x = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

private_bar_plot(
    x,
    private_count_max3Day_week,
    "CASE 2: Differentially private - Visitors Count per Max 3 Days in Week",
    "Week days",
    "Visitors",
)

# Comparison between private and non-private count

In [ ]:
# Case 1 Comparison
# if error occurs loading - reload total_visitors_day cell above
labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
x1 = total_visitors_day
x2 = private_count_perDay_week

comparison_graph(
    labels,
    x1,
    x2,
    "count_perDay_week",
    "private_count_perDay_week",
    "Visitors",
    "CASE 1: Visitors Count per Day VS Differentially private - Visitors Count per Day",
)

In [ ]:
# Case 2 Comparison
labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
x1 = total_visitors_max3days
x2 = private_count_max3Day_week

comparison_graph(
    labels,
    x1,
    x2,
    "count_perDay_week",
    "private_count_perDay_max3week",
    "CASE 2: Visitors",
    "Visitors Count per Day VS Differentially private - Visitors Count per Day",
)

## 3. Sum-up revenue per day of the week

The previous example demonstrates how the contributed partitions are bounded. Now, you will demonstrate how individual contributions are clamped. Imagine Alice decides to calculate the sum of the restaurant revenue per week day in a differentially private way. For this she needs to sum up the visitors’ daily spending at the restaurant. For simplicity, let’s assume a visitor enters the restaurant **at most once a day but multiple times a week.**

In [ ]:
# MAX_CONTRIBUTED_DAYS = 4
MIN_EUROS_SPENT = 0

MAX_EUROS_SPENT = 50

epsilon = math.log(3)

Create new dataframe and set the per day entries to "0" using **set_visitor_count0()** same as in the previous case followed by changing the 0's to 1's for the visitor count using **per-day_visits()**. To find the sum-up revenue per day of the week, instead of visitor count, money spent by the visitor is needed. Using **per_day_amount()**  the amount spent by the visitors for that day replaces the day count i.e., 1's. Followed by **max_partitions_contributed()** to convert the visit_contributions to "True" for  max contributions equal to 4. Finally, the sum-up revenue per day of the week with max contributions equal to 4 is obtained by removing the extra contributions using **remove_extra_contributions()**. 

In [ ]:
revenue_df = pd.DataFrame({"VisitorId": visitor_df["VisitorId"].unique()})

set_visitor_count0(revenue_df).head()  # returns revenue_df set days values to 0

In [ ]:
per_day_visits(
    revenue_df
).head()  # returns revenue_df days values - 1 -> present 0 -> absent

In [ ]:
# Add Per Day Amount to visitor_count_df
# Replace the Day attendance with the amount spent on that day


def per_day_amount(df):
    for index_no in visitor_df.index:
        day_no = visitor_df.loc[index_no, "Day"]
        id_no = visitor_df.loc[index_no, "VisitorId"]
        amount = visitor_df.loc[index_no, "Money spent (euros)"]
        # index_val = visitor_df.loc[visitor_df["VisitorId"] == id_no].index[0]
        # print("index val : ",index_val, "day_no :", day_no)

        # other dataframe
        visitor_index_val = df.loc[visitor_count_df["VisitorId"] == id_no].index[0]
        # print("New index val : ",visitor_index_val)
        visitor_day_val = df.loc[visitor_index_val, "Day_" + str(day_no)]
        # print("Day val : ",visitor_day_val)
        df.at[visitor_index_val, "Day_" + str(day_no)] = amount

    return df


per_day_amount(revenue_df).head()

In [ ]:
max_partitions_contributed(
    4, revenue_df
).head()  # returns revenue_df with True and False values for max contribution = 4

In [ ]:
# Remove extra contribution from visitors_Maxweek DataFrame
# returns visitors_Maxweek
remove_extra_contributions(4, revenue_df).head()

### Non-private per day amount

In [ ]:
# Calculate per day amount - Unbounded sum
# To check the difference between unbounded and bounded sum
per_day_amount = []

for day in days:  # day = day_1
    amount = revenue_df.loc[:, day].sum()
    per_day_amount.append(amount)


print(days)
print(per_day_amount)

### To calculate the private per day amount using PyDP library

In [ ]:
# Calculate Differentially private amount
def private_per_day_amount_bounded(lt):
    x = BoundedSum(epsilon=epsilon, lower_bound=MIN_EUROS_SPENT, upper_bound=MAX_EUROS_SPENT, l0_sensitivity=4)  # MAX_CONTRIBUTED_DAYS
    return x.quick_result(lt)

### Both Private and Non-private values

In [ ]:
bounded = []
private_bounded = []

for day in days:

    amount_bounded = []

    for index_no in revenue_df.index:
        amount = revenue_df.loc[index_no, day]

        if amount > MAX_EUROS_SPENT:
            amount = MAX_EUROS_SPENT

        if amount != 0:
            amount_bounded.append(amount)

    total = private_per_day_amount_bounded(amount_bounded)
    total1 = sum(amount_bounded)
    private_bounded.append(total)
    bounded.append(total1)

print(days)
print("Private Values:", private_bounded)
print("Original Values:", bounded)

In [ ]:
# bar_plot(x,y,title,x_label,y_label)
x = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

bar_plot(x, bounded, "Amount per day of week - Bounded", "Week days", "Visitors")

In [ ]:
# private_bar_plot(x,y,title,x_label,y_label)
x = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

private_bar_plot(
    x,
    private_bounded,
    "Differentially Private Amount per day of week - Bounded",
    "Week days",
    "Visitors",
)

In [ ]:
# Comparison graph

labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
x1 = bounded
x2 = private_bounded

comparison_graph(
    labels,
    x1,
    x2,
    "perDayAmountBounded",
    "privatePerDayAmountBounded",
    "Visitors",
    "Amount Spent Per Day VS Differentially private - Amount Spent Per Day",
)